In [5]:
import requests
page = requests.get('https://www.basketball-reference.com/players/a/')
page

<Response [200]>

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [31]:
results = soup.find_all('th', class_="left")
str(results[0])

'<th class="left" data-append-csv="abdelal01" data-stat="player" scope="row"><a href="/players/a/abdelal01.html">Alaa Abdelnaby</a></th>'

In [26]:
import re

In [29]:
pattern = 'data-append-csv="\w*\d\d"'

In [38]:
match = re.findall(pattern, str(results[0]))
type(match)

list

In [42]:
match = re.findall(r'\w*\d\d', str(match))

In [46]:
name = ''.join(match)

In [47]:
names = []
for result in results:
    result = str(result)
    match = re.findall(r'data-append-csv="\w*\d\d"', result)
    match = re.findall(r'\w*\d\d', str(match))
    name = ''.join(match)
    names.append(name)

In [50]:
dfs = []

for name in names:
    df = pd.read_html(f'https://www.basketball-reference.com/players/a/{name}.html')[0]
    dfs.append(df)

dfa = pd.concat(dfs)

XMLSyntaxError: no text parsed from document (<string>, line 0)